# Testing Agentic approach to rubric evaluation

In [185]:
import openai
import glob, os, time, json, re, http, asyncio
from pydantic import BaseModel, Field
from enum import Enum
from agents import Agent, Runner, Tool, trace
from pydantic import BaseModel, Field
from enum import Enum
from typing import List

# Escape Codes para imprimir mensajes con color
Rd = "\033[1;31m"
Grn = "\033[1;32m"
Ylw = "\033[1;33m"
Blu = "\033[1;34m"
Mag = "\033[1;35m"
Rst = "\033[0m"

spacer = f"{80*'-'}"

In [186]:
def pretty_print(data, indent=0):
    """
    Recursively prints nested dictionaries, lists, and strings in a readable format.
    
    Args:
        data: The data structure to print (dict, list, str, etc.).
        indent: The current indentation level (used for nested structures).
    """
    # Define the indentation string
    spacing = "    " * indent

    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{Grn}{spacing}📌{key}{Rst}:")
            pretty_print(value, indent + 1)
    elif isinstance(data, list):
        for index, item in enumerate(data):
            print(f"{Mag}{spacing}➡️ [{index}]{Rst}")
            pretty_print(item, indent + 1)
    else:
        # Print strings or other primitive types
        print(f"{spacing}{data}")

Rubric Evaluation Agent

In [187]:
from typing import Union

class EvidenceStrength(str, Enum):
    EXPLICIT_STRONG = "Explicit & Strong Evidence"
    EXPLICIT_WEAK = "Explicit but Weak Evidence"
    IMPLICIT = "Implicit Evidence or Assumption"
    NONE = "No Evidence Provided"
    NOT_REQUIRED = "No Evidence Required (Trivial or Common Knowledge)"

class NotFoundEvidenceRubric(BaseModel):
    """Formato para evidencia faltante, que indica el campo específico y el motivo por el cual no se completó."""
    field: str = Field(description="Campo específico de la rúbrica que falta por completar.")
    motivo: str = Field(description="Motivo por el cual no se completó el campo. Puede ser 'No se encontró evidencia' o 'No aplica'.")


class FoundEvidence(BaseModel):
    """Formato de evidencia encontrada, que indica si es claro y sustentado con evidencia."""
    judgement: str = Field(description=("Juicio sobre si se cumple con la rúbrica y por qué. Debe incluir una evaluación clara y fundamentada, indicando si las evidencias presentadas son suficientes, claras, y relevantes para cumplir con los criterios establecidos."))
    evidence: List[str] = Field(description=("Extracto exacto del texto que justifica el criterio evaluado. Debe ser un fragmento tal cual como viene en el texto provisto. Si hay varias evidencias, sepáralas por bullets o numeración."))
    clarity: str = Field(description="Indica si la evidencia es clara y coherente o si le falta claridad. Ej: 'Claridad adecuada' o 'Falta de explicación clara'.")
    referencia: EvidenceStrength = Field(description=(
            "Referencia o sustento científico de la evidencia. Puede ser cita bibliográfica, inferencia, "
            "conocimiento asumido, evidencia débil, o trivial. Las opciones posibles son:\n"
            "1. 'Explicit & Strong Evidence': Evidencia clara y robusta, con fuente confiable.\n"
            "2. 'Explicit but Weak Evidence': Fuente directa, pero débil o ambigua.\n"
            "3. 'Implicit Evidence or Assumption': Basado en conocimiento tácito o asumido, sin cita directa.\n"
            "4. 'No Evidence Provided': No se presenta evidencia respaldatoria.\n"
            "5. 'No Evidence Required (Trivial or Common Knowledge)': Conocimiento común, no requiere evidencia."
        ))
    manifestation: str = Field(description=(
    "Forma en que se manifiesta la evidencia dentro del documento evaluado. "
    "Las formas más comunes son:\n"
    "- 'Afirmación textual directa': Una afirmación explícita encontrada en el texto.\n"
    "- 'Hipervínculo': Enlace incluido en el texto que dirige a evidencia externa.\n"
    "- 'Cita bibliográfica': Referencia académica o técnica incluida como fuente.\n"
    "- 'Documento adjunto o anexo': Mención de un documento externo relacionado.\n"
    "- 'Otro': En caso de que la evidencia adopte una forma no convencional (especificar)."
))

class TechnicalMeritRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar el mérito técnico del proyecto."""
    
    technical_evidence: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description=(
            "Evidencias del criterio relacionadas con la base técnica del proyecto. "
            "Debe incluir pruebas de la solidez técnica, enfoque metodológico, y coherencia de los fundamentos "
            "que sustentan el proyecto. Ejemplos incluyen modelos técnicos, análisis de viabilidad, "
            "y detalles sobre las tecnologías empleadas. Si se encuentra evidencia, usar el esquema FoundEvidence. "
            "Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
        )
    )
    scientific_justification: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description=(
            "Evidencias relacionadas con la justificación científica que respalda el proyecto. "
            "Debe incluir la validación científica, teorías aplicadas, estudios previos relevantes, y el uso "
            "de referencias académicas que apoyan la propuesta. Ejemplos incluyen citas bibliográficas, "
            "resultados de investigaciones previas, o documentación que valida la base científica del proyecto. "
            "Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
        )
    )

class TechnicalSolutionRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la claridad y coherencia en la solución técnica propuesta."""
    
    solution_clarity: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias relacionadas con la claridad de la solución propuesta para abordar el problema. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    solution_coherence: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias relacionadas con la coherencia de la solución y su alineación con el problema. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    justification: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que sustentan la justificación de la solución técnica propuesta. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )

class TechnologyMaturityRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar el nivel de maduración tecnológica según la escala TRL."""
    
    technology_maturity_level: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen el nivel de maduración tecnológica del proyecto según la escala TRL. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    evidence_of_progress: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias de los avances hacia el cumplimiento de TRL 4 y de los pasos realizados hacia un TRL superior. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    validation_evidence: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la validación realizada a la tecnología en las distintas etapas de desarrollo. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    plan_for_advancement: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias de los planes estructurados para avanzar a TRL 5 o superior, con pasos claros y definidos. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )

class ImplementationFeasibilityRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la viabilidad de la implementación considerando recursos y contexto."""
    
    implementation_feasibility: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen la viabilidad de la implementación en función de los recursos disponibles y el contexto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    resource_availability: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la disponibilidad de recursos necesarios para la implementación del proyecto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    risk_assessment: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen los riesgos asociados con la implementación y las estrategias para mitigar estos riesgos. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    execution_strategy: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen la estrategia clara de ejecución del proyecto, con pasos bien definidos y cronograma adecuado. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    uncertainty_management: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que muestran cómo se gestionan las incertidumbres y áreas de mejora dentro del proyecto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )

class TeamCompositionRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la experiencia y trayectoria del equipo proponente en proyectos similares."""
    
    team_experience: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen la experiencia del equipo en proyectos similares y su capacidad técnica. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    relevant_background: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre el historial y antecedentes relevantes de los miembros del equipo en áreas clave. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    team_complementarity: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que muestran la complementariedad de las habilidades dentro del equipo, asegurando una cobertura adecuada de todas las áreas necesarias para el proyecto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    leadership_capabilities: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran las capacidades de liderazgo del equipo, incluyendo la gestión y la toma de decisiones. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    individual_expertise: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que destacan la experiencia individual de los miembros clave en el equipo, especialmente en áreas críticas para el proyecto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )

class BudgetJustificationRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la distribución y justificación del presupuesto asignado al proyecto."""
    
    budget_clarity: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran que la distribución del presupuesto está claramente definida y alineada con los objetivos del proyecto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    resource_distribution: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la distribución de los recursos del presupuesto de forma coherente y equilibrada entre las diferentes necesidades del proyecto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    justification_of_expenses: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que muestran cómo se justifica cada gasto dentro del presupuesto, asegurando que cada partida tiene un propósito claro y relevante para el proyecto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    optimization_opportunities: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que evalúan si el presupuesto ha sido optimizado adecuadamente o si existen áreas donde se podrían reducir gastos sin afectar el proyecto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    budget_alignment: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que confirman que el presupuesto está alineado con los objetivos y la estrategia del proyecto, asegurando que los recursos se destinan donde son más necesarios. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    completeness: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran que el presupuesto está completamente detallado, sin omisiones ni partidas faltantes. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )

class ExpectedImpactsMonitoringRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la definición de impactos esperados y su metodología de monitoreo."""
    
    impact_definition: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran que los impactos esperados del proyecto están claramente definidos, detallados y alineados con los objetivos del proyecto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    impact_relevance: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que muestran que los impactos esperados están directamente relacionados con el problema a resolver, y cómo la solución propuesta contribuirá de manera significativa. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    monitoring_methodology: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la metodología de monitoreo propuesta para seguir el progreso de los impactos. Esto incluye métodos de seguimiento, indicadores y herramientas que se usarán. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    monitoring_plan: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias de que el plan de monitoreo es detallado y cubre las etapas necesarias del proyecto, incluyendo tiempos y formas de medición de los impactos. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    impact_assessment: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que indican cómo se evaluarán los impactos esperados, incluyendo cómo se medirá su éxito y qué métricas se utilizarán para su evaluación. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    adjustment_mechanism: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen los mecanismos de ajuste en caso de que los impactos no se logren según lo planeado, incluyendo acciones correctivas o alternativas. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )

class WorkPlanRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar el plan de trabajo propuesto con hitos y cronograma."""
    
    work_plan_structure: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran que el plan de trabajo está bien estructurado, con una clara descripción de las tareas, los entregables y las fases del proyecto. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    milestones_definition: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que muestran que los hitos del proyecto están claramente definidos, con objetivos alcanzables y medibles para cada fase. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    timeline_and_schedule: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description=(
            "Evidencias que detallan el cronograma del proyecto, indicando los tiempos estimados para cada fase o tarea, "
            "y si son realistas y alcanzables. Nota: Si el cronograma proviene de un documento escaneado por OCR, "
            "es posible que esté distorsionado o incompleto. En caso de sospecha, levantar el comentario correspondiente. "
            "Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
        )
    )
    task_distribution: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran cómo se distribuyen las tareas entre los miembros del equipo y cómo se planifica el uso de recursos en función de los hitos. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    risk_and_contingency_planning: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la identificación de posibles riesgos y la existencia de un plan de contingencia para mitigarlos y asegurar el cumplimiento de los hitos. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    deliverable_quality: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que confirman que los entregables del proyecto están bien definidos, tienen criterios claros de calidad y son evaluables. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )

class IntellectualPropertyRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la propiedad intelectual en proceso o concedida."""
    
    ip_generation_potential: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran el potencial de generación de propiedad intelectual en el proyecto, como innovaciones tecnológicas, procesos, o productos originales. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    ip_protection_evidence: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias de que se ha realizado algún avance en la protección de la propiedad intelectual, como solicitudes de patentes, marcas, derechos de autor, u otros mecanismos de protección. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    ip_trademark_or_patent_status: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias del estado actual de las solicitudes de protección intelectual, como patentes o marcas registradas, y si están en trámite o si ya han sido concedidas. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    ip_management_strategy: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la existencia de una estrategia clara para la gestión y protección de la propiedad intelectual, como planificación, análisis de mercado, o acuerdos de licencia. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )
    ip_granted_or_advanced_status: List[Union[FoundEvidence, NotFoundEvidenceRubric]] = Field(
        ...,
        description="Evidencias que indican que la propiedad intelectual ha sido concedida o está en proceso avanzado, mostrando la protección definida para los derechos relacionados. Si se encuentra evidencia, usar el esquema FoundEvidence. Si falta evidencia, usar el esquema NotFoundEvidenceRubric."
    )


class MaturityRubric(BaseModel):
    """Formato de Rúbrica, para juntar las evidencias relacionadas a la madurez de un proyecto científico."""
    # Criterio de mérito técnico
    merito_tecnico: TechnicalMeritRubicCriterion = Field(
        description="Evidencias relacionadas a la base técnica del proyecto y su justificación científica."
    )
    
    # Criterio de solución técnica
    solucion_tecnica: TechnicalSolutionRubicCriterion = Field(
        description="Evidencias relacionadas a la claridad y coherencia de la solución técnica propuesta para abordar el problema."
    )
    
    # Criterio de madurez tecnológica
    madurez_tecnologica: TechnologyMaturityRubicCriterion = Field(
        description="Evidencias relacionadas al nivel de maduración tecnológica del proyecto según la escala TRL."
    )
    
    # Criterio de viabilidad de implementación
    viabilidad_implementacion: ImplementationFeasibilityRubicCriterion = Field(
        description="Evidencias relacionadas a la viabilidad de implementación considerando recursos y contexto."
    )
    
    # Criterio de composición del equipo
    equipo_proponente: TeamCompositionRubicCriterion = Field(
        description="Evidencias relacionadas a la experiencia y trayectoria del equipo en proyectos similares."
    )
    
    # Criterio de justificación del presupuesto
    justificacion_presupuesto: BudgetJustificationRubicCriterion = Field(
        description="Evidencias relacionadas a la distribución y justificación del presupuesto asignado al proyecto."
    )
    
    # Criterio de impactos esperados y su metodología de monitoreo
    impactos_monitoreo: ExpectedImpactsMonitoringRubicCriterion = Field(
        description="Evidencias relacionadas a la definición clara de los impactos esperados y su monitoreo."
    )
    
    # Criterio de plan de trabajo propuesto con hitos y calendario
    plan_trabajo: WorkPlanRubicCriterion = Field(
        description="Evidencias relacionadas a la claridad del plan de trabajo con entregables y cronograma."
    )
    
    # Criterio de propiedad intelectual en proceso o concedida
    propiedad_intelectual: IntellectualPropertyRubicCriterion = Field(
        description="Evidencias relacionadas a la existencia o potencial de generación de propiedad intelectual."
    )

# Rubric Evaluation

In [188]:
rubric_evaluator_agent = Agent(
    name="Rubric Evaluator Agent",
    instructions="""
    Description:
    The Rubric Evaluator Agent conducts a rigorous initial evaluation of a project proposal based on a structured maturity assessment rubric. Its core function is to extract, assess, and organize **hard evidence** from the proposal, aligning it to each rubric criterion. This agent prioritizes **thoroughness and honesty**: it does **not assume or infer evidence** and is **not obligated to complete all rubric fields** if no solid evidence exists.

    Approach:
    - **Extensively search for explicit, traceable evidences** related to each criterion in the maturity rubric. It is possible that you might find more than one evidence for each criterion. Might be possible you find no evidence at all.
    - **Avoid speculative judgments**: if there is no clear or strong evidence, the agent must **leave the field unscored or flag it accordingly**.
    - If evidence is strong, the agent should **confidently acknowledge it**. If evidence is weak, limited, or ambiguous, this should also be made clear.
    - **If the evidence is NOT found, explicitly flag it as missing.**

    Inputs:
    1. A project or proposal document containing technical, methodological, organizational, and contextual information.
    2. A predefined maturity rubric with clearly described criteria (e.g., TRL, technical merit, feasibility, impacts, team experience).

    Output:
    A structured object (`MaturityRubric`) where each field corresponds to a rubric criterion and contains an evidence-grounded analysis. The output:
    - Highlights **whether the criterion is strongly supported, weakly supported, or unsupported**.
    - May leave fields empty if **no verifiable evidence is found**.

    Guidelines:
    - The agent **does not evaluate based on potential**—only on present, explicit information.
    - The output should include direct references to the text or reasoning showing **why** a criterion is (or isn't) fulfilled.
    - Do **not be afraid to say the evidence is insufficient**, vague, or absent.
    - **Flag missing evidence explicitly.**
    
    Considerations:
    - The information is provided from an OCR scan of a PDF document. Don't hesitate to highlight that maybe some evidences are there but there might be importation issues. Clasically, this is the case with tables or images (chronograms for instance).
    
    Workflow Position:
    This agent works **prior to** the deliberation phase with other agents (e.g., Proponent, Devil’s Advocate). It establishes a **fact-based baseline**, grounded in evidence, that serves to inform and constrain further discussion.

    Adjustment:
    The agent can revise its output if new documentation is submitted or clarification is provided later in the evaluation process.
    
    Rubric Format:
    
    # Rúbrica de Evaluación de Proyectos de Investigación

    ## 1. Mérito técnico
    _Evaluación de la base científica y técnica del proyecto._

    - **5 puntos**: Justificación técnica excelente, con respaldo sólido y claridad.
    - **4 puntos**: Base técnica bien sustentada.
    - **3 puntos**: Justificación aceptable, aunque con áreas de mejora.
    - **2 puntos**: Fundamento débil o poco claro.
    - **1 punto**: Justificación técnica vaga o sin referencias claras.
    - **0 puntos**: Sin justificación técnica.

    ---

    ## 2. Solución técnica propuesta
    _Claridad, coherencia e innovación de la solución planteada al problema._

    - **5 puntos**: Solución innovadora, bien formulada y alineada al problema.
    - **4 puntos**: Solución clara y estructurada, con justificación sólida.
    - **3 puntos**: Solución adecuada pero con áreas poco desarrolladas.
    - **2 puntos**: Solución identificable, pero con poca claridad o detalle.
    - **1 punto**: Planteamiento vago o no relacionado con el problema.
    - **0 puntos**: Sin una solución clara o coherente.

    ---

    ## 3. Nivel de madurez tecnológica (TRL ≥ 4 preferente)
    _Estado de desarrollo tecnológico del proyecto._

    - **5 puntos**: TRL 4 con evidencia sólida y plan claro hacia TRL 5 o más.
    - **4 puntos**: TRL 4 validado en laboratorio.
    - **3 puntos**: TRL 4 con validación parcial.
    - **2 puntos**: TRL 3 con intención de avanzar a TRL 4, validación limitada.
    - **1 punto**: TRL 3 sin evidencia de avance.
    - **0 puntos**: TRL menor a 3 o sin evidencia.

    ---

    ## 4. Viabilidad de implementación
    _Factibilidad técnica y operativa con los recursos disponibles._

    - **5 puntos**: Proyecto claramente viable, con estrategia de ejecución.
    - **4 puntos**: Implementación bien fundamentada, con riesgos controlables.
    - **3 puntos**: Viabilidad aceptable, pero con incertidumbres.
    - **2 puntos**: Posible, pero con riesgos altos sin mitigación clara.
    - **1 punto**: Dificultades significativas para implementarse.
    - **0 puntos**: No viable en las condiciones actuales.

    ---

    ## 5. Equipo proponente (perfil y trayectoria)
    _Experiencia del equipo en proyectos similares._

    - **5 puntos**: Equipo altamente calificado, con trayectoria sólida y liderazgo.
    - **4 puntos**: Equipo con experiencia y capacidades complementarias.
    - **3 puntos**: Equipo adecuado, con áreas de mejora en trayectoria.
    - **2 puntos**: Algún miembro con experiencia, pero equipo débil.
    - **1 punto**: Experiencia limitada o poco clara.
    - **0 puntos**: Sin experiencia ni evidencia de capacidad técnica.

    ---

    ## 6. Justificación del presupuesto
    _Coherencia entre los recursos solicitados y los objetivos._

    - **5 puntos**: Presupuesto detallado, óptimo y bien justificado.
    - **4 puntos**: Presupuesto estructurado y alineado a los objetivos.
    - **3 puntos**: Presupuesto aceptable, con oportunidades de optimización.
    - **2 puntos**: Justificación parcial, con áreas no justificadas.
    - **1 punto**: Distribución poco clara o inconsistente.
    - **0 puntos**: Presupuesto sin justificación o con errores graves.

    ---

    ## 7. Impactos esperados y monitoreo
    _Definición clara de impactos y su seguimiento._

    - **5 puntos**: Impactos sólidos con monitoreo definido y estructurado.
    - **4 puntos**: Impactos bien definidos con monitoreo adecuado.
    - **3 puntos**: Impactos razonables, con metodología mejorable.
    - **2 puntos**: Impactos definidos, pero monitoreo débil.
    - **1 punto**: Impactos mencionados sin relación clara con el problema.
    - **0 puntos**: No se definen impactos ni forma de medirlos.

    ---

    ## 8. Plan de trabajo (hitos y calendario)
    _Claridad del cronograma y entregables._

    - **5 puntos**: Plan detallado, con hitos definidos y calendario realista.
    - **4 puntos**: Plan estructurado, con hitos claros y alcanzables.
    - **3 puntos**: Hitos definidos, pero calendario poco claro.
    - **2 puntos**: Plan poco estructurado o incompleto.
    - **1 punto**: Plan desorganizado o sin entregables definidos.
    - **0 puntos**: No presenta plan de trabajo.

    ---

    ## 9. Propiedad intelectual (preferente)
    _Potencial o existencia de generación de propiedad intelectual._

    - **5 puntos**: Propiedad intelectual concedida o proceso avanzado y claro.
    - **4 puntos**: En trámite, con estrategia clara.
    - **3 puntos**: Avances hacia protección identificados.
    - **2 puntos**: Posible generación, pero sin evidencia clara.
    - **1 punto**: Bajo potencial de generación.
    - **0 puntos**: No se considera o no hay intención.
    """,
    tools=[],
    output_type=MaturityRubric,
    model="gpt-4o-mini-2024-07-18",
)



In [189]:
with open('./QRO/779_Aguadelluvia.txt', 'rb') as file:
    file_content = file.read()
    file_content = file_content.decode('utf-8', errors='ignore')

In [190]:
with trace("Prueba de Extracción de Evidencias"):
    result = await Runner.run(
        rubric_evaluator_agent,
        input=f"Cumple con tu rol de trabajo, responde en español. El contenido viene a continuación: \n{file_content}",
        )


In [191]:
pretty_print(result.final_output.model_dump())

📌merito_tecnico:
    📌technical_evidence:
        ➡️ [0]
            📌judgement:
                La justificación técnica es sólida, presentando un enfoque innovador en la integración de tecnologías para la captación y tratamiento de agua de lluvia. Se mencionan modelos de predicción de precipitación y el uso de reactores electroquímicos de oxidación avanzada.
            📌evidence:
                ➡️ [0]
                    El proyecto plantea el desarrollo de dos instrumentos tecnológicos complementarios cuya integración permitirá generar un plan viable de aprovechamiento eficiente del agua de lluvia en el estado de Querétaro.
                ➡️ [1]
                    Se han desarrollado reactores que eliminan contaminantes cuyas pruebas han sido evaluadas a nivel laboratorio y en condiciones reales de operación.
            📌clarity:
                Claridad adecuada
            📌referencia:
                EvidenceStrength.EXPLICIT_STRONG
            📌manifestation:
              

# Evaluation Stage

In [192]:
from agents import function_tool

@function_tool
def original_document() -> str:
    """Provides the original document, that desires funding from our institution."""
    return file_content

@function_tool
def rubric_facts() -> MaturityRubric:
    """Provides the list of found facts that align to the rubric. Contains evidence extracted from the original document. Done previously by an evaluator."""
    return result.final_output

@function_tool
def rubric_format() -> str:
    """Thie is the rubric for evluation."""
    text = """
    # Rúbrica de Evaluación de Proyectos de Investigación

    ## 1. Mérito técnico
    _Evaluación de la base científica y técnica del proyecto._

    - **5 puntos**: Justificación técnica excelente, con respaldo sólido y claridad.
    - **4 puntos**: Base técnica bien sustentada.
    - **3 puntos**: Justificación aceptable, aunque con áreas de mejora.
    - **2 puntos**: Fundamento débil o poco claro.
    - **1 punto**: Justificación técnica vaga o sin referencias claras.
    - **0 puntos**: Sin justificación técnica.

    ---

    ## 2. Solución técnica propuesta
    _Claridad, coherencia e innovación de la solución planteada al problema._

    - **5 puntos**: Solución innovadora, bien formulada y alineada al problema.
    - **4 puntos**: Solución clara y estructurada, con justificación sólida.
    - **3 puntos**: Solución adecuada pero con áreas poco desarrolladas.
    - **2 puntos**: Solución identificable, pero con poca claridad o detalle.
    - **1 punto**: Planteamiento vago o no relacionado con el problema.
    - **0 puntos**: Sin una solución clara o coherente.

    ---

    ## 3. Nivel de madurez tecnológica (TRL ≥ 4 preferente)
    _Estado de desarrollo tecnológico del proyecto._

    - **5 puntos**: TRL 4 con evidencia sólida y plan claro hacia TRL 5 o más.
    - **4 puntos**: TRL 4 validado en laboratorio.
    - **3 puntos**: TRL 4 con validación parcial.
    - **2 puntos**: TRL 3 con intención de avanzar a TRL 4, validación limitada.
    - **1 punto**: TRL 3 sin evidencia de avance.
    - **0 puntos**: TRL menor a 3 o sin evidencia.

    ---

    ## 4. Viabilidad de implementación
    _Factibilidad técnica y operativa con los recursos disponibles._

    - **5 puntos**: Proyecto claramente viable, con estrategia de ejecución.
    - **4 puntos**: Implementación bien fundamentada, con riesgos controlables.
    - **3 puntos**: Viabilidad aceptable, pero con incertidumbres.
    - **2 puntos**: Posible, pero con riesgos altos sin mitigación clara.
    - **1 punto**: Dificultades significativas para implementarse.
    - **0 puntos**: No viable en las condiciones actuales.

    ---

    ## 5. Equipo proponente (perfil y trayectoria)
    _Experiencia del equipo en proyectos similares._

    - **5 puntos**: Equipo altamente calificado, con trayectoria sólida y liderazgo.
    - **4 puntos**: Equipo con experiencia y capacidades complementarias.
    - **3 puntos**: Equipo adecuado, con áreas de mejora en trayectoria.
    - **2 puntos**: Algún miembro con experiencia, pero equipo débil.
    - **1 punto**: Experiencia limitada o poco clara.
    - **0 puntos**: Sin experiencia ni evidencia de capacidad técnica.

    ---

    ## 6. Justificación del presupuesto
    _Coherencia entre los recursos solicitados y los objetivos._

    - **5 puntos**: Presupuesto detallado, óptimo y bien justificado.
    - **4 puntos**: Presupuesto estructurado y alineado a los objetivos.
    - **3 puntos**: Presupuesto aceptable, con oportunidades de optimización.
    - **2 puntos**: Justificación parcial, con áreas no justificadas.
    - **1 punto**: Distribución poco clara o inconsistente.
    - **0 puntos**: Presupuesto sin justificación o con errores graves.

    ---

    ## 7. Impactos esperados y monitoreo
    _Definición clara de impactos y su seguimiento._

    - **5 puntos**: Impactos sólidos con monitoreo definido y estructurado.
    - **4 puntos**: Impactos bien definidos con monitoreo adecuado.
    - **3 puntos**: Impactos razonables, con metodología mejorable.
    - **2 puntos**: Impactos definidos, pero monitoreo débil.
    - **1 punto**: Impactos mencionados sin relación clara con el problema.
    - **0 puntos**: No se definen impactos ni forma de medirlos.

    ---

    ## 8. Plan de trabajo (hitos y calendario)
    _Claridad del cronograma y entregables._

    - **5 puntos**: Plan detallado, con hitos definidos y calendario realista.
    - **4 puntos**: Plan estructurado, con hitos claros y alcanzables.
    - **3 puntos**: Hitos definidos, pero calendario poco claro.
    - **2 puntos**: Plan poco estructurado o incompleto.
    - **1 punto**: Plan desorganizado o sin entregables definidos.
    - **0 puntos**: No presenta plan de trabajo.

    ---

    ## 9. Propiedad intelectual (preferente)
    _Potencial o existencia de generación de propiedad intelectual._

    - **5 puntos**: Propiedad intelectual concedida o proceso avanzado y claro.
    - **4 puntos**: En trámite, con estrategia clara.
    - **3 puntos**: Avances hacia protección identificados.
    - **2 puntos**: Posible generación, pero sin evidencia clara.
    - **1 punto**: Bajo potencial de generación.
    - **0 puntos**: No se considera o no hay intención.

    """
    return text

In [193]:
from typing import Union

class MissingRubric(BaseModel):
    field: str = Field(description="Campo específico de la rúbrica que falta por completar.")
    motivo: str = Field(description="Motivo por el cual no se completó el campo. Puede ser 'No se encontró evidencia' o 'No aplica'.")

class MaturityRubric(BaseModel):
    """Formato de Rúbrica, para juntar las evidencias relacionadas a la madurez de un proyecto científico."""
    # Criterio de mérito técnico
    merito_tecnico: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la base técnica del proyecto y su justificación científica. Comentarios sobre la solidez técnica y científica. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    solucion_tecnica: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la claridad y coherencia de la solución técnica propuesta para abordar el problema. Comentarios sobre la solución técnica. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    madurez_tecnologica: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas al nivel de maduración tecnológica del proyecto según la escala TRL. Comentarios sobre el nivel de madurez tecnológica. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    viabilidad_implementacion: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la viabilidad de implementación considerando recursos y contexto. Comentarios sobre la factibilidad de implementación. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    equipo_proponente: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la experiencia y trayectoria del equipo en proyectos similares. Comentarios sobre la composición del equipo. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    justificacion_presupuesto: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la distribución y justificación del presupuesto asignado al proyecto. Comentarios sobre la justificación del presupuesto. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    impactos_monitoreo: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la definición clara de los impactos esperados y su monitoreo. Comentarios sobre los impactos esperados y su monitoreo. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    plan_trabajo: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la claridad del plan de trabajo con entregables y cronograma. Comentarios sobre el plan de trabajo. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    propiedad_intelectual: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la existencia o potencial de generación de propiedad intelectual. Comentarios sobre la propiedad intelectual. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )

class Evaluation(BaseModel):
    pitch: str = Field(description="El pitch del proyecto, que se desea evaluar.")
    project_description: str = Field(description="¿Qué consiste el proyecto de manera elaborada? ¿Cuáles son los diferenciadores?.")
    antecedentes: str = Field(description="¿Cuáles son los antecedentes del proyecto? ¿Qué ha hecho el proponente/equipo antes? Si ellos están proponiendo algo nuevo que no han ejectuado, indicarlo, es importante.")
    rubric: MaturityRubric = Field(description="Opinión sobre la rúbrica de evaluación, que contiene los criterios y evidencias encontradas. ¿Está completo o falta?")

class Observation(BaseModel):
    """Observación sobre la evaluación."""
    original: str = Field(description="Extracto de la observación original, que servirá como contexto para tu análisis. Asegúrate de incluir suficiente información para que sea comprensible.")
    observation: str = Field(description="Escribe tu observación, apoyo, crítica o refutación sobre la retroalimentación conferida. Sé claro, conciso y fundamenta tu respuesta.")

class Agree(str, Enum):
    STRONGLY_DISAGREE = "❌❌ Strongly Disagree"
    DISAGREE = "❌ Disagree"
    AGREE = "✔️ Agree"
    STRONGLY_AGREE = "✔️✔️ Strongly Agree"

class Observation_to_critique(BaseModel):
    """Observación sobre la evaluación de otros."""
    reviewer: str = Field(description="Escribe el nombre del reviewer quien estás criticando o apoyando.")
    agree: Agree = Field(description="¿Estás de acuerdo con la observación del colega?")
    original: str = Field(description="Extracto de la observación original, que servirá como contexto para tu análisis. Asegúrate de incluir suficiente información para que sea comprensible.")
    observation: str = Field(description="Escribe tu observación, apoyo, crítica o refutación sobre la retroalimentación conferida. Sé claro, conciso y fundamenta tu respuesta.")

class Observations(BaseModel):
    """Observaciones sobre la evaluación."""
    observations: List[Observation] = Field(description="Escribe la observación, apoyo crítica o refutación sobre la retroalimentación conferida de acuerdo a tu rol.")

class Observations_critiques(BaseModel):
    """Observaciones sobre la evaluación."""
    observations: List[Observation_to_critique] = Field(description="Escribe la observación, apoyo crítica o refutación sobre la retroalimentación conferida de acuerdo a tu rol.")
    
class Project_Manager_Evaluation(BaseModel):
    """Evaluación del Project Manager."""
    plan: List[Observation] = Field(
        description=(
            "Observaciones relacionadas al plan del proyecto. Evalúa si el plan está bien estructurado, "
            "con hitos claros, cronograma realista y entregables definidos. Identifica posibles áreas de mejora "
            "en la organización y planificación del proyecto."
        )
    )
    recurso_tiempo: List[Observation] = Field(
        description=(
            "Observaciones sobre la gestión de tiempo del proyecto. Analiza si los tiempos asignados a cada tarea "
            "son realistas y alcanzables. Identifica posibles riesgos de retrasos y su impacto en el cronograma general."
        )
    )
    recurso_dinero: List[Observation] = Field(
        description=(
            "Observaciones sobre la gestión de dinero del proyecto. Evalúa si el presupuesto está bien distribuido "
            "y justificado. Asegúrate de que los recursos financieros se asignen a las áreas críticas del proyecto, "
            "considerando posibles omisiones o gastos innecesarios. Verifica si los salarios cumplen con estándares "
            "mínimos (e.g., salario mínimo mexicano) y si hay áreas donde el presupuesto podría optimizarse."
        )
    )
    viability: List[Observation] = Field(
        description=(
            "Observaciones sobre la viabilidad del proyecto desde múltiples ángulos. Evalúa si el proyecto es "
            "realizable considerando los recursos disponibles, el contexto operativo, y los riesgos asociados. "
            "Incluye análisis sobre la claridad de los objetivos, la alineación con las capacidades del equipo, "
            "la disponibilidad de infraestructura, y la factibilidad técnica. Identifica posibles problemas como "
            "falta de recursos, riesgos no mitigados, dependencias externas críticas, o cronogramas poco realistas. "
            "Si el proyecto está bien estipulado y creíble, se debe destacar su solidez y viabilidad."
        )
    )

class Business_Designer_Evalaution(BaseModel):
    """Evaluación del Business Designer."""	
    commercial_viability: List[Observation] = Field(
        description=(
            "Observaciones sobre la viabilidad comercial del proyecto. Evalúa si el proyecto tiene un mercado "
            "definido, una propuesta de valor clara y un modelo de negocio sostenible. Identifica posibles riesgos "
            "comerciales y su impacto en la viabilidad del proyecto."
        )
    )
    market_necessity: List[Observation] = Field(
        description=(
            "Observaciones sobre la necesidad del mercado para el proyecto. Evalúa si el proyecto aborda un problema "
            "real y relevante en el mercado. Identifica posibles áreas de mejora en la identificación de necesidades "
            "del cliente y su alineación con la solución propuesta."
        )
    )
    market_timing: List[Observation] = Field(
        description=(
            "Observaciones sobre el momento del mercado para el proyecto. Evalúa si el mercado está saturado con demasiada "
            "competencia y bajos rendimientos, o si es un mercado novedoso con alto valor agregado. Incluye comentarios sobre "
            "si el mercado está consolidado con grandes jugadores o si aún no lo está."
        )
    )
    impact: List[Observation] = Field(
        description=(
            "Observaciones sobre el impacto del proyecto. Evalúa si el proyecto tiene un impacto positivo en la "
            "sociedad, el medio ambiente o la economía. Identifica posibles áreas de mejora en la medición y "
            "evaluación del impacto del proyecto."
        )
    )

class RubricScore(BaseModel):
    """Evaluación de cada sección de la rúbrica"""
    campo: str = Field(description="Campo de la rúbrica que se está evaluando. Tal y como aparece en el documento de la rúbrica (e.g.: Estadp actual de la tecnología (TRL preferente 4)")
    score: int = Field(description="Puntuación asignada al campo de la rúbrica. Debe ser un número entre 0 y 5.")
    score_as_it_appears_in_rubric: int = Field(description="Este es el score y el nombre del score asignado tal como viene en el documento (e.g: **2 puntos**: TRL 3 con intención de avanzar a TRL 4, validación limitada.)")
    pos_comment: List[str] = Field(description="Comentarios positivos sobre la puntuación asignada. Deben ser claros y concisos. No estás comprometido a siempre emitir un comentario.")
    neg_comment: List[str] = Field(description="Comentarios en contra sobre la puntuación asignada. Deben ser claros y concisos. Si el score no es 5 (perfecto), estás comprometido a dar una buena razón de por qué no.")
    score_rationale: str = Field(description="Razonamiento detrás de la puntuación asignada. Debe ser claro y conciso, explicando por qué se asignó esa puntuación en particular. Si mencionas que faltaron detalles, dí específicamente cuáles. Es retroalimentación importante.")

class RubricScores(BaseModel):
    """Formato de Rúbrica, para juntar las evidencias relacionadas a la madurez de un proyecto científico."""
    # Criterio de mérito técnico
    merito_tecnico: RubricScore = Field(
        description="Se dirige a evidencias relacionadas a la base técnica del proyecto y su justificación científica. Comentarios sobre la solidez técnica y científica. Debe ser un objeto RubricScore."
    )
    
    solucion_tecnica: RubricScore = Field(
        description="Se dirige a evidencias relacionadas a la claridad y coherencia de la solución técnica propuesta para abordar el problema. Comentarios sobre la solución técnica. Debe ser un objeto RubricScore."
    )
    
    madurez_tecnologica: RubricScore = Field(
        description="Se dirige a evidencias relacionadas al nivel de maduración tecnológica del proyecto según la escala TRL. Comentarios sobre el nivel de madurez tecnológica. Debe ser un objeto RubricScore."
    )
    
    viabilidad_implementacion: RubricScore = Field(
        description="Se dirige a evidencias relacionadas a la viabilidad de implementación considerando recursos y contexto. Comentarios sobre la factibilidad de implementación. Debe ser un objeto RubricScore."
    )
    
    equipo_proponente: RubricScore = Field(
        description="Se dirige a evidencias relacionadas a la experiencia y trayectoria del equipo en proyectos similares. Comentarios sobre la composición del equipo. Debe ser un objeto RubricScore."
    )
    
    justificacion_presupuesto: RubricScore = Field(
        description="Se dirige a evidencias relacionadas a la distribución y justificación del presupuesto asignado al proyecto. Comentarios sobre la justificación del presupuesto. Debe ser un objeto RubricScore."
    )
    
    impactos_monitoreo: RubricScore = Field(
        description="Se dirige a evidencias relacionadas a la definición clara de los impactos esperados y su monitoreo. Comentarios sobre los impactos esperados y su monitoreo. Debe ser un objeto RubricScore."
    )
    
    plan_trabajo: RubricScore = Field(
        description="Se dirige a evidencias relacionadas a la claridad del plan de trabajo con entregables y cronograma. Comentarios sobre el plan de trabajo. Debe ser un objeto RubricScore."
    )
    
    propiedad_intelectual: RubricScore = Field(
        description="Se dirige a evidencias relacionadas a la existencia o potencial de generación de propiedad intelectual. Comentarios sobre la propiedad intelectual. Debe ser un objeto RubricScore."
    )



class Consolidated_Evaluation(BaseModel):
    """Evaluación consolidada."""
    consolidated_project_pitch: Evaluation = Field(description="Evaluación del proyecto obtenida leyendo a partir del documento y agregando las observaciones de los evaluadores.")
    fortalezas_de_propuesta: List[str] = Field(description="A partir del documento y de la discusión, se obtienen las fortalezas de la propuesta. ¿Cuáles son los puntos fuertes del planteamiento del proyecto?")
    debilidades_de_propuesta: List[str] = Field(description="A partir del documento y de la discusión, se obtienen las debilidades de la propuesta. ¿Cuáles son los puntos débiles del planteamiento del proyecto?")
    recomendaciones: List[str] = Field(description="A partir del documento y de la discusión, se obtienen las recomendaciones. ¿Cuáles son las recomendaciones para mejorar el planteamiento del proyecto?")
    score_rubrica: RubricScores = Field(description="Evaluación detallada de cada sección de la rúbrica, asignando puntuaciones y comentarios específicos para cada criterio. Proporciona un análisis integral del desempeño del proyecto en relación con los criterios establecidos.")
        

In [194]:
visionary_domain_specialist = Agent(
    name="Visionary Domain Specialist",
    instructions="""
    instructions:
    The Visionary Domain Specialist Agent is tasked with identifying the specific domain or field of expertise required for the proposal by analyzing the original document, rubric format, and rubric facts. This agent acts as a domain specialist, adapting its focus based on the content and context of the document. As a visionary evaluator for the company, the agent's objective is to create value by identifying and supporting projects with high potential for success and innovation.

    Inputs:
    1. The original document containing the proposal, background, and context.
    2. The rubric format outlining the evaluation criteria.
    3. The rubric facts providing additional context or guidelines for evaluation.

    Objective:
    1. Analyze the document to determine the relevant domain or field of expertise (e.g., technology, healthcare, education, etc.).
    2. Provide insights and recommendations specific to the identified domain, ensuring that the proposal aligns with best practices, trends, and innovations in that field.
    3. Highlight domain-specific strengths, opportunities, and potential for value creation within the proposal.
    4. Identify weaknesses or areas for improvement, offering constructive feedback to enhance the proposal's quality and alignment with the rubric criteria.
    5. Advocate for the proposal by emphasizing its visionary aspects and potential to drive innovation and success.

    Adjust to Feedback:
    1. If additional context or clarification is provided, refine the domain analysis and recommendations accordingly.
    2. Adapt the critique and insights based on new information or adjustments to the proposal.
    3. Be prepared to defend the generated feedback if questioned, providing justification and evidence for the positions taken.

    Output:
    The agent's output will include:
    - Strengths: Key aspects of the proposal that align with the rubric and demonstrate high potential for success.
    - Weaknesses: Areas where the proposal could be improved, with specific suggestions for enhancement.

    Workflow Position:
    This agent operates early in the evaluation process, setting the stage for domain-specific considerations that other agents may build upon. The agent's visionary perspective ensures that promising projects are given the attention and support needed to maximize their potential. The feedback provided will be cross-examined later in the evaluation process.
    """,
    tools=[original_document, rubric_format, rubric_facts],
    model="gpt-4o-2024-08-06",
    output_type=Evaluation,
)

realist_domain_specialist = Agent(
    name="Realist Domain Specialist",
    instructions="""
    instructions:
    The Realist Domain Specialist Agent operates as a devil's advocate within the specific domain or field identified in the proposal. This agent's primary role is to critically evaluate the proposal by questioning its value and feasibility, considering the stakes involved if the project is approved. The agent grounds its analysis in the current state of the field, identifying how things are done today and using this as a baseline to challenge the proposal's assumptions, innovations, and potential impact.

    Inputs:
    1. The original document containing the proposal, background, and context.
    2. The rubric format outlining the evaluation criteria.
    3. The rubric facts providing additional context or guidelines for evaluation.

    Objective:
    1. Identify the current state of the domain or field as described in the document, including standard practices, methodologies, and benchmarks.
    2. Critically assess the proposal's claims, questioning its value, feasibility, and alignment with the current state of the field.
    3. Highlight potential risks, weaknesses, or oversights in the proposal, particularly where it diverges from established practices or fails to address key challenges.
    4. Provide constructive feedback to refine the proposal, ensuring it addresses the stakes and challenges inherent in its implementation.

    Adjust to Feedback:
    1. If additional context or clarification is provided, refine the critique and recommendations accordingly.
    2. Adapt the analysis based on new information or adjustments to the proposal.
    3. Be prepared to defend the critique if questioned, providing justification and evidence for the positions taken.

    Output:
    The agent's output will include:
    - A detailed analysis of the current state of the field, serving as a baseline for evaluation.
    - A critique of the proposal's value and feasibility, with specific references to risks, weaknesses, and areas for improvement.
    - Constructive feedback aimed at enhancing the proposal's alignment with the domain's realities and addressing potential stakes.

    Workflow Position:
    This agent operates early in the evaluation process, providing a critical perspective that challenges the proposal's assumptions and ensures that its value and feasibility are rigorously tested. The feedback provided will be cross-examined later in the evaluation process.
    """,
    tools=[original_document, rubric_format, rubric_facts],
    model="gpt-4o-2024-08-06",
    output_type=Evaluation,
)

project_manager_agent = Agent(
    name="Project Manager Agent",
    instructions="""
    instructions:
    The Project Manager Agent evaluates the proposal exclusively from a project management perspective. This agent focuses on assessing timelines, budgets, resource allocation, and overall project execution capabilities. It does not delve into technical or domain-specific aspects but ensures that the project is feasible and well-structured from a management standpoint.

    Inputs:
    1. The proposal document, including timelines, budgets, and resource plans.
    2. Rubric outlining project management criteria (e.g., milestones, deliverables, risk management, resource allocation).

    Objective:
    1. Evaluate the clarity and feasibility of the proposed timeline, ensuring that milestones and deliverables are realistic and achievable.
    2. Assess the budget, verifying whether it aligns with the scope of the project and whether resource allocation is sufficient and appropriate.
    3. Identify potential risks related to project execution, such as resource shortages, unrealistic deadlines, or insufficient contingency planning.
    4. Provide recommendations to improve project management aspects, such as refining timelines, reallocating resources, or enhancing risk mitigation strategies.

    Adjust to Feedback:
    1. If additional details or clarifications are provided, refine the evaluation to reflect the updated information.
    2. Adapt recommendations based on new insights or adjustments to the project plan.

    Output:
    The agent's output will include:
    - A detailed assessment of the project timeline, budget, and resource allocation.
    - Identification of risks and areas for improvement in project management.
    - Constructive feedback to enhance the feasibility and structure of the project from a management perspective.

    Workflow Position:
    This agent operates early in the evaluation process, ensuring that the project is well-grounded in sound project management principles. Its feedback serves as a foundation for assessing the project's overall viability and execution potential.
    """,
    tools=[original_document, rubric_format, rubric_facts],
    model="gpt-4o-2024-08-06",
    output_type=Project_Manager_Evaluation,
)

business_designer_agent = Agent(
    name="Business Designer Agent",
    instructions="""
    Description:
    The Business Designer Agent evaluates the proposal from a business design perspective, focusing on the value proposition, business model, market fit, and scalability. This agent ensures that the proposal aligns with sound business principles and has the potential to create sustainable value.

    Inputs:
    1. The proposal document, including details on the value proposition, target market, revenue model, and scalability plans.
    2. Rubric outlining business design criteria (e.g., market analysis, competitive advantage, customer segmentation, revenue streams).

    Objective:
    1. Assess the clarity and strength of the value proposition, ensuring it addresses a real market need or problem.
    2. Evaluate the business model, including revenue streams, cost structure, and scalability potential.
    3. Analyze the target market and customer segmentation, identifying whether the proposal demonstrates a strong understanding of its audience.
    4. Identify potential risks or weaknesses in the business design, such as unclear revenue models, limited market potential, or scalability challenges.
    5. Provide constructive feedback to refine the business aspects of the proposal, ensuring alignment with best practices and market trends.

    Adjust to Feedback:
    1. If additional details or clarifications are provided, refine the evaluation to reflect the updated information.
    2. Adapt recommendations based on new insights or adjustments to the business model or market strategy.

    Output:
    The agent's output will include:
    - A detailed assessment of the value proposition, business model, and market fit.
    - Identification of risks and areas for improvement in the business design.
    - Constructive feedback to enhance the proposal's business viability and scalability.

    Workflow Position:
    This agent operates early in the evaluation process, ensuring that the proposal is grounded in a strong business foundation. Its feedback serves as a basis for assessing the proposal's potential for long-term success and sustainability.
    """,
    tools=[original_document, rubric_format, rubric_facts],
    model="gpt-4o-2024-08-06",
    output_type=Project_Manager_Evaluation,
)

orchestator_agent = Agent(
    name="Orchestator Agent",
    instructions="""
    Description:
    The Orchestrator Agent consolidates the knowledge, observations, and concerns provided by all previous agents to produce a comprehensive and balanced evaluation of the proposal. Its primary goal is to synthesize the value proposition, assess its adherence to the rubric, and provide a final list of strengths and weaknesses. This agent ensures that the company invests in projects with strong foundations and high potential for success.

    Approach:
    - Gather and analyze inputs from all agents, including their observations, critiques, and recommendations.
    - Synthesize a clear and concise value proposition based on the consolidated feedback.
    - Evaluate how well the proposal adheres to the rubric, highlighting areas of alignment and misalignment.
    - Identify and prioritize strengths and weaknesses, discarding opinions that are overly controversial, convoluted, or unsupported by evidence.
    - Maintain a critical yet constructive perspective, ensuring that the final evaluation supports sound decision-making.
    - When scores aren't perfect, explicitly state the reasons. If there are missing specifics, clearly identify which ones are missing. Avoid vague feedback, as detailed and actionable feedback is essential for applicants.

    Inputs:
    1. Consolidated outputs from all agents, including their observations, critiques, and recommendations.
    2. The original proposal document and rubric for reference.

    Objective:
    1. Write down the consolidated value proposition, ensuring it reflects the core strengths and potential of the proposal.
    2. Assess the proposal's adherence to the rubric, providing a balanced evaluation of its alignment with the criteria.
    3. Enlist the final list of strengths and weaknesses, prioritizing clarity, relevance, and actionable insights.
    4. Discard overly controversial or convoluted opinions that do not contribute to a constructive evaluation.
    5. Ensure the final output supports the company's goal of investing in projects with strong foundations and high potential for success.

    Output:
    The agent's output will include:
    - A consolidated value proposition.
    - An evaluation of the proposal's adherence to the rubric.
    - A prioritized list of strengths and weaknesses, with actionable insights.

    Workflow Position:
    This agent operates at the final stage of the evaluation process, synthesizing all prior inputs to provide a comprehensive and balanced assessment. Its output serves as the basis for the company's decision-making, ensuring that only well-founded and high-potential projects are pursued.
    """,
    tools=[original_document, rubric_format, rubric_facts],
    model="gpt-4o-2024-08-06",
    output_type=Consolidated_Evaluation,
)


## Step 1: Evaluation

In [195]:
import asyncio

msg = ("Tienes acceso a documentos clave: la rúbrica de evaluación, las evidencias extraídas de la rúbrica, y el documento original de la propuesta. "
    "Tu tarea es analizar la propuesta con un enfoque equilibrado: identificar proyectos con alto potencial de innovación que puedan generar valor para la empresa, "
    "pero también ser crítico y objetivo, ya que estamos invirtiendo nuestros propios recursos. "
    "Evalúa cuidadosamente los riesgos y oportunidades, asegurándote de que las decisiones estén fundamentadas en evidencia sólida y alineadas con los criterios de la rúbrica.")

with trace("MAAF Evaluation"):
    s1_DomSpecVisionary, s1_DomSpecRealist, s1_ProjManager, s1_BizDesigner = await asyncio.gather(
        Runner.run(
            visionary_domain_specialist,
            input=f"{msg}.",
            ),
        Runner.run(
            realist_domain_specialist,
            input=f"{msg}.",
            ),
        Runner.run(
            project_manager_agent,
            input=f"{msg}.",
            ),
        Runner.run(
            business_designer_agent,
            input=f"{msg}.",
            ),
    
    )


In [196]:
print(spacer)
print(f"{Ylw}Visionary Domain Specialist{Rst}")
pretty_print(s1_DomSpecVisionary.final_output.model_dump())

print(spacer)
print(f"{Ylw}Realist Domain Specialist{Rst}")
pretty_print(s1_DomSpecRealist.final_output.model_dump())

print(spacer)
print(f"{Ylw}Project Manager{Rst}")
pretty_print(s1_ProjManager.final_output.model_dump())

print(spacer)
print(f"{Ylw}Business Designer{Rst}")
pretty_print(s1_BizDesigner.final_output.model_dump())

--------------------------------------------------------------------------------
Visionary Domain Specialist
📌pitch:
    El proyecto busca desarrollar tecnología para aprovechar eficientemente el agua de lluvia en Querétaro mediante un sistema de predicción de precipitaciones y reactores de oxidación electroquímica, promoviendo la economía circular del agua.
📌project_description:
    El proyecto consiste en crear un sistema integrado para el aprovechamiento del agua de lluvia en Querétaro, usando pronósticos meteorológicos y tecnología de oxidación electroquímica avanzada. Este enfoque modular y ecológico pretende reducir el consumo de agua potable mediante la reutilización de agua tratada, todo alineado con las prácticas de sostenibilidad actuales.
📌antecedentes:
    El equipo ha trabajado previamente en modelos meteorológicos globales y regionales y ha desarrollado reactores de oxidación evaluados en laboratorio y en condiciones reales, contando con apoyo financiero de importantes or

In [ ]:
evaluation_content = {"role": "assistant", 
     "content": [
        {"agent": "Domain Specialist 1", "content": s1_DomSpecRealist.final_output},
        {"agent": "Domain Specialist 2", "content": s1_DomSpecRealist.final_output},
        {"agent": "Project Manager", "content": s1_ProjManager.final_output},
        {"agent": "Business Designer", "content": s1_BizDesigner.final_output},
         ]}

discussion = result.to_input_list()
discussion.append(evaluation_content)

## Cross-critique Round

In [198]:
# Step 3: Cross-Critique Round

msg = f"""
You previously examined a proposal and provided feedback. Now, you will cross-examine the comments of other agents regarding the initially provided document. Your task is to critically evaluate their observations **only if you feel strongly against the observation**. If you do not have a strong opinion, you are not obligated to respond.

Focus on:
1. Identifying any gaps, inconsistencies, or unsupported claims in their feedback if you strongly disagree.
2. Providing constructive and specific reasoning to support your stance.
3. Remember that we evaluate so our company can catch good proposals to mature. But also having in mind that we can't support ill-founded ones.

The discussion so far is provided below for your review:

{discussion}
"""

visionary_domain_specialist.output_type = Observations_critiques
realist_domain_specialist.output_type = Observations_critiques 
project_manager_agent.output_type = Observations_critiques
business_designer_agent.output_type = Observations_critiques

s2_DomSpecVisionary, s2_DomSpecRealist, s2_ProjManager, s2_BizDesigner  = await asyncio.gather(
    Runner.run(
        visionary_domain_specialist,
        input=msg + "\nNOTE: You are Domain Specialist 1 in the evidences. Don't evaluate yourself.",
        ),
    Runner.run(
        realist_domain_specialist,
        input=msg + "\nNOTE: You are Domain Specialist 2 in the evidences. Don't evaluate yourself.",
        ),
    Runner.run(
        project_manager_agent,
        input=msg + "\nNOTE: You are Project Manager in the evidences. Don't evaluate yourself.",
        ),
    Runner.run(
        business_designer_agent,
        input=msg + "\nNOTE: You are Business Designer in the evidences. Don't evaluate yourself.",
        )
)

In [199]:
print(spacer)
print(f"{Ylw}Visionary Domain Specialist{Rst}")
pretty_print(s2_DomSpecVisionary.final_output.model_dump())

print(spacer)
print(f"{Ylw}Realist Domain Specialist{Rst}")
pretty_print(s2_DomSpecRealist.final_output.model_dump())

print(spacer)
print(f"{Ylw}Project Manager{Rst}")
pretty_print(s2_ProjManager.final_output.model_dump())

print(spacer)
print(f"{Ylw}Business Designer{Rst}")
pretty_print(s2_BizDesigner.final_output.model_dump())

--------------------------------------------------------------------------------
Visionary Domain Specialist
📌observations:
    ➡️ [0]
        📌reviewer:
            Project Manager
        📌agree:
            Agree.DISAGREE
        📌original:
            El plan es innovador al integrar modelos de predicción y tecnología de tratamiento de agua de manera modular. Sin embargo, sería beneficioso incluir un plan más detallado sobre cómo se gestionarán los recursos en cada etapa del proyecto.
        📌observation:
            La observación es válida en cuanto a que un plan más detallado puede proporcionar claridad. Sin embargo, el enfoque modular propuesto ya considera una gestión escalonada de recursos, permitiendo ajustes conforme se avanza. El diseño actual está pensado para adaptarse a cambios y optimizar recursos a lo largo del ciclo del proyecto.
    ➡️ [1]
        📌reviewer:
            Project Manager
        📌agree:
            Agree.DISAGREE
        📌original:
            El cro

In [200]:
evaluation_content = {"role": "assistant", 
     "content": [
        {"agent": "Domain Specialist 1", "content": s2_DomSpecRealist.final_output},
        {"agent": "Domain Specialist 1", "content": s2_DomSpecRealist.final_output},
        {"agent": "Project Manager", "content": s2_ProjManager.final_output},
        {"agent": "Business Designer", "content": s2_BizDesigner.final_output},
         ]}

discussion.append(evaluation_content)

## Consolidation Stage

In [201]:
s4_Orchestrator = await asyncio.gather(
    Runner.run(
        orchestator_agent,
        input=(
            "You are the Orchestrator Agent. Your task is to consolidate the knowledge, observations, and concerns provided by all previous agents to produce a comprehensive and balanced evaluation of the proposal. "
            "Your primary goal is to synthesize the value proposition, assess its adherence to the rubric, and provide a final list of strengths and weaknesses. "
            "You must evaluate the proposal strictly according to the rubric (consult it carefully). "
            "Additionally, it is necessary to address the discussion provided, including the identified fields, observations, and critiques from the specialists. "
            "If you believe some comments are irrelevant or unsupported, you may choose to exclude them, but justify your reasoning. "
            "Keep in mind that our role is to identify promising projects worth taking the risk of funding, while avoiding reckless decisions that could lead to failure. "
            "Here is the discussion so far: "
            f"{discussion}"
        ),
    )
)

In [202]:
print(spacer)
print(f"{Ylw}Orchestrator{Rst}")
pretty_print(s4_Orchestrator[0].final_output.model_dump())

--------------------------------------------------------------------------------
Orchestrator
📌consolidated_project_pitch:
    📌pitch:
        El proyecto busca desarrollar un sistema integrado que combine la predicción de precipitaciones con tecnología electroquímica avanzada para captación, potabilización y tratamiento de agua de lluvia y agua gris en Querétaro, mejorando la gestión hídrica de la región mediante un esquema de economía circular.
    📌project_description:
        El proyecto presenta la creación de dos instrumentos tecnológicos clave que permitirán el desarrollo de un plan estratégico para la captación y tratamiento de agua de lluvia. Se trata de un sistema de predicción meteorológica y la implementación de reactores electroquímicos de oxidación avanzada. La solución propuesta se integra en un esquema modular que proporciona agua potable y reutiliza agua gris, promoviendo la economía circular y adaptándose a diferentes condiciones locales, incluso en áreas rurales sin 

In [205]:
discussion

[{'content': 'Cumple con tu rol de trabajo, responde en español. El contenido viene a continuación: \n--- Page 1 ---\r\n \r\n1 \r\n\r\n \r\n\r\n\r\n--- Page 2 ---\r\n02 Tabla de Contenido \r\n \r\n \r\n\r\n01.-  \r\nTítulo  \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n  1 \r\n\r\n02.-  \r\nTabla de contenido \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n  2 \r\n\r\n03.-  \r\nResumen ejecutivo \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n  3 \r\n\r\n04.-  \r\nEquipo de trabajo \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n  4 \r\n\r\n05.-  \r\nCarta de participación de la institución colaboradora \r\n \r\n \r\n  6 \r\n\r\n06.-  \r\nAnálisis del estado del arte \r\n \r\n \r\n \r\n \r\n \r\n \r\n  7 \r\n\r\n07.-  \r\nObjetivo general y objetivos específicos \r\n \r\n \r\n \r\n \r\n  8  \r\n\r\n08.-  \r\nAntecedentes sobre la viabilidad técnica \r\n \r\n \r\n \r\n \r\n10 \r\n\r\n09.-  \r\nDescripción técnica de la propuesta  \r\n \r\n \r\n \r\n \r\n14 \r\n\r\n10.-  \r\nPropuesta de valor y factores diferenciad